# Modèle statique

In [1]:
import numpy as np
import pandas as pd
import altair as alt

## Méthode

On utilise un modèle de pluviométrie statique : on sait exactement, et à l'avance, quand la pluie tombera et en quelle quantité. 

Pour construire le modèle nous utilisons la fiche climatique de Clermont-Ferrand qui donne, pour chaque mois, la hauteur cumulée tombée, et le nombre de jours où il a plu. 

Comme on ne sait pas quel jour il a plu, on distribue aléatoirement les jours de pluie dans le mois, et on considère qu'il tombe toujours la même quantité. 

## Modèle de pluviométrie

### Pluviométrie locale

Considérons la fiche climatique de Clermont-Ferrand. 

In [2]:
moyenne_clermont = pd.read_csv('../../../data/clermont_2.csv', sep=',', )
precipitations = pd.read_csv('../../../data/avg_yearly_rain_clermont.csv',
                              sep=',').replace(np.nan, 0)

FileNotFoundError: [Errno 2] No such file or directory: '../../../data/clermont_2.csv'

Et visualisons-la :

In [ ]:
alt.Chart(moyenne_clermont).mark_bar().encode(
    alt.X('mois:T', title='Mois'),
    alt.Y('hauteur', title='Hauteur')
)

alt.Chart(...)

Simulons maintenant des pluies quotidiennes.

In [ ]:
alt.Chart(precipitations).mark_bar().encode(
    x=alt.X('doy:T', 
            title = 'Jour de l\'annee', 
            axis=alt.Axis(labelFontSize=0,
                           labelOpacity=0)),
    y=alt.Y('mm:Q', title = 'Hauteur des précipitations')
).properties(width=450)



alt.Chart(...)

Comme prévu, ce modèle n'est pas représentatif de la réalité, car les pluies sont bien trop homogènes en hauteur. 
On en tire immédiatement l'eau disponible à chaque pluie grâce à la surface du toit. 

## Consommation

### Consommation potagère

La consommation est décrite dans une section précédente. 


## Dimensionnement cuve

Dans un premier temps, voyons la hauteur cumulée d'eau tombée. 

In [ ]:
precipitations['cumul']=np.cumsum(precipitations.mm)
alt.Chart(precipitations).mark_area(
    color="lightblue",
    interpolate='step-after',
    line=True
).encode(
    x=alt.X('doy', title = "Jour de l'année", axis = alt.Axis(labelOpacity=0)),
    y=alt.Y('cumul', title = "Pluie cumulée depuis le début de l'année")
).properties(width=600)

alt.Chart(...)

Et maintenant regardons les besoins.

In [ ]:
besoins = pd.read_csv('../../../data/consommation_potager.csv', sep=',')

Quels sont les besoins totaux ? 

In [ ]:
print(np.sum(besoins['conso_potager']*100)) 


81677.0


Le potager a donc besoin, en tout, de 82 $m^3$ au cours d'une année. 
Mais une partie de ces besoins est fournie par la pluie, lorsqu'elle tombe. 

En première approximation, considérons qu'à chaque fois qu'il pleut les besoins sont nuls pour la journée, sans influer sur ceux du lendemain. 


In [ ]:
df = pd.merge(precipitations, besoins, on='doy')
df.head()

,doy,month,mm,cumul,julianDay,conso_potager,conso_paillage
0,01-01,1,0.0,0.0,1,0.0,0.0
1,01-02,1,0.0,0.0,2,0.0,0.0
2,01-03,1,0.0,0.0,3,0.0,0.0
3,01-04,1,0.0,0.0,4,0.0,0.0
4,01-05,1,0.0,0.0,5,0.0,0.0


In [ ]:
df['irrigationSurfacique'] = df.conso_potager - df.mm
df.irrigationSurfacique[df['irrigationSurfacique']<0]=0

/var/folders/x9/h8vwd75d3352d6ngzx4sqgj00000gn/T/ipykernel_838/2816986552.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.irrigationSurfacique[df['irrigationSurfacique']<0]=0


In [ ]:
alt.Chart(df).mark_line().encode(
    x = alt.X('doy', title="Jour de l'année", axis=alt.Axis(labelOpacity=0)),
    y = alt.Y('irrigationSurfacique', title="Besoins en irrigation")
).properties(width=450).interactive()

alt.Chart(...)

In [ ]:
df['besoinsTotaux'] = df.irrigationSurfacique * 100
df['apportsTotaux'] = df.mm*200
# besoins totaux donne la totalité du besoin pour le potager, soit sur 100 m2

On accède donc, via la surface du potager et celle du toit, à l'ensemble des besoins et l'ensemble des apports quotidiens. 


### Enfin, la cuve

In [ ]:
cuve = [0]
for p in [0, 1000, 3000, 20000, 0]:
    cuve = [p]
    for index in range(len(df)):
        cuve.append(cuve[index-1] + df['apportsTotaux'].iloc[index] - df['besoinsTotaux'].iloc[index])
    print('cuve ', p, ', min = ', min(cuve), ', max = ', max(cuve))
    print('****')
    print('diff = ', abs(min(cuve)-max(cuve)))
    print('****')

    

cuve  0 , min =  0 , max =  29186.0
****
diff =  29186.0
****
cuve  1000 , min =  1000 , max =  30186.0
****
diff =  29186.0
****
cuve  3000 , min =  3000 , max =  32186.0
****
diff =  29186.0
****
cuve  20000 , min =  20000 , max =  49186.0
****
diff =  29186.0
****
cuve  0 , min =  0 , max =  29186.0
****
diff =  29186.0
****


In [ ]:
print('remplissage cuve au premier mai :', cuve[120])
print('remplissage au premier octobre :', cuve[270] )

remplissage cuve au premier mai : 12258.0
remplissage au premier octobre : 18919.0


On a visiblement besoin de la dérivée. 

In [ ]:
dcuve =[[],[]]
for index in range(len(cuve)):
    dcuve.append([cuve[index], 0])


In [ ]:
dcuve = pd.DataFrame(list(zip(range(len(cuve)), cuve)), 
                     index = range(len(cuve)),
                     columns=['doy', 'volume'])

In [ ]:
alt.Chart(dcuve).mark_line().encode(
    x='doy',
    y='volume'
).interactive()

alt.Chart(...)

On peut donc calculer, à partir de ces valeurs, le volume minimal de la cuve, c'est à dire celui qui permet de faire face à la plus forte diminution de volume pendant la saison de végétation. 
 

In [ ]:
max(dcuve.loc[dcuve['doy'].isin(range(120,270)), "volume"])-min(dcuve.loc[dcuve['doy'].isin(range(120,270)), "volume"])

12067.0

On arrive à la valeur de 12 mètres cubes. 

Montrons l'évolution du volume d'une cuve de 12 m3

In [ ]:
cuve=[0]
vmax = 9000
for index in range(len(df)):
        cuve.append(cuve[index-1] + df['apportsTotaux'].iloc[index] - df['besoinsTotaux'].iloc[index])
        if cuve[index] > vmax:
                cuve[index] = vmax

dcuve = pd.DataFrame(list(zip(range(len(cuve)), cuve)), 
                     index = range(len(cuve)),
                     columns=['doy', 'volume'])

alt.Chart(dcuve).mark_line().encode(
        x='doy',
        y='volume'
)

alt.Chart(...)

Avec paillage : 
Les besoins sont bien inférieurs. 
